In [1]:
!pip install -q ultralytics deep-sort-realtime

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 88.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 101.6 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 76.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━

In [2]:
import cv2
import os
from ultralytics import YOLO
from deep_sort_realtime.deepsort_tracker import DeepSort

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
model_path = "/kaggle/input/input-files/best.pt"
model = YOLO(model_path) 

In [4]:
tracker = DeepSort(max_age=15, n_init=4, nn_budget=100)

In [5]:
input_video = "/kaggle/input/input-files/15sec_input_720p.mp4"
output_video = "/kaggle/working/re_ID_output.mp4"

In [6]:
cap = cv2.VideoCapture(input_video)
fps = cap.get(cv2.CAP_PROP_FPS)
w, h = int(cap.get(3)), int(cap.get(4))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video, fourcc, fps, (w, h))

In [7]:
class_colors = {
    "player": (255, 255, 255),      # White
    "goalie": (0, 255, 0),          # Green
    "goalkeeper": (0, 255, 0),      # Green
    "referee": (0, 255, 255),       # Yellow
    "football": (0, 0, 255)         # Red
}

In [8]:
def draw_styled_player(frame, track_id, bbox, label, role_color):
    x1, y1, x2, y2 = map(int, bbox)
    cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
    radius = int(0.4 * ((x2 - x1 + y2 - y1) / 2))
    ellipse_radius = max(10, radius // 2)

    # Semi-ellipse
    cv2.ellipse(frame, (cx, cy), (radius, 10), 0, 0, 180, role_color, 2)

    # ID label
    cv2.rectangle(frame, (cx - 20, cy - radius - 30), (cx + 50, cy - radius - 5), role_color, -1)
    cv2.putText(frame, f"{label[0].upper()}#{track_id}", (cx - 15, cy - radius - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)

In [9]:
print("Processing video...")
frame_count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Run YOLOv11 prediction
    results = model.predict(source=frame, conf=0.6, iou=0.5, stream=True)

    detections = []
    detection_metadata = []

    for r in results:
        for box in r.boxes.data.cpu().numpy():
            x1, y1, x2, y2, conf, cls = box
            label = model.names[int(cls)].lower()

            # Filter detections
            if label in ["player", "goalkeeper", "referee"] and conf > 0.6:
                x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
                if x2 - x1 > 20 and y2 - y1 > 20:  # skip small noise
                    cropped_img = frame[y1:y2, x1:x2]
                    detections.append(([x1, y1, x2 - x1, y2 - y1], conf, cropped_img))
                    detection_metadata.append((label, (x1, y1, x2, y2)))

    # Track objects
    tracks = tracker.update_tracks(detections, frame=frame)

    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        bbox = list(map(int, track.to_ltrb()))

        # Match metadata for class label
        for label, ref_bbox in detection_metadata:
            if abs(bbox[0] - ref_bbox[0]) < 20 and abs(bbox[1] - ref_bbox[1]) < 20:
                color = class_colors.get(label.lower(), (255, 255, 255))
                draw_styled_player(frame, track_id, bbox, label, color)
                break

    out.write(frame)

print(f"Finished processing {frame_count} frames.")
cap.release()
out.release()
print(f"Re-ID video saved to: {output_video}")

Processing video...

0: 384x640 12 players, 1 referee, 59.7ms
Speed: 6.2ms preprocess, 59.7ms inference, 241.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 players, 1 referee, 55.5ms
Speed: 3.1ms preprocess, 55.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 football, 14 players, 1 referee, 55.5ms
Speed: 1.9ms preprocess, 55.5ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 football, 14 players, 1 referee, 55.5ms
Speed: 1.6ms preprocess, 55.5ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 football, 14 players, 1 referee, 55.5ms
Speed: 1.8ms preprocess, 55.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 players, 1 referee, 40.0ms
Speed: 1.7ms preprocess, 40.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 14 players, 1 referee, 38.9ms
Speed: 1.7ms preprocess, 38.9ms inference, 1.6ms postprocess per i